In [1]:
# Install Environment dependencies
!pip install hugsvision
# DONT LOAD WHAT IS NOT NEEDED
#!pip install pyvips
#!sudo apt-get -y install lynx
#!sudo apt-get -y install w3m
#!sudo apt-get install desktop-file-utils

#import modules
import pandas as pd
import os
from pathlib import Path
from hugsvision.dataio.VisionDataset import VisionDataset
from torchvision.transforms import InterpolationMode
InterpolationMode.BICUBIC
import shutil
import glob

os.environ["WANDB_DISABLED"] = "true"

# ONLY IMPORT WHAT IS NEEDED
#import tensorflow as tf
#from PIL import Image
#import matplotlib.pyplot as plt
#import cv2 as cv
#import PIL
#import numpy as np
#import torch
#import glob
#from skimage import io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for pycocotools: filename=pycocotools-2.0.4-cp37-cp37m-linux_x86_64.whl size=370081 sha256=6590b1d2dd4fa964d53a6e2a9e90fff575de88d06fff53de2a5253b8b78cf145
  Stored in directory: /root/.cache/pip/wheels/a3/5f/fa/f011e578cc76e1fc5be8dce30b3eb9fd00f337e744b3bba59b
Successfully built pycocotools


/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [2]:
df_train1 = pd.read_csv('../input/mayo-clinic-1024-jpg-part1/train_with_groups.csv')
df_train2 = pd.read_csv('../input/mayo-clinic-1024-jpg-part2-1/train_with_groups.csv')
df_train3 = pd.read_csv('../input/mayo-clinic-1024-jpg-part3/train_with_groups.csv')
df_train4 = pd.read_csv('../input/mayo-clinic-1024-jpg-part4-1/train_with_groups.csv')
df_train5 = pd.read_csv('../input/mayo-clinic-1024-jpg-part5-1/train_with_groups.csv')
df_train6 = pd.read_csv('../input/mayo-clinic-1024-jpg-part6/train_with_groups.csv')
df_train7 = pd.read_csv('../input/mayo-clinic-1024-jpg-part7-1/train_with_groups.csv')
df_train8 = pd.read_csv('../input/mayo-clinic-1024-jpg-part8/train_with_groups.csv')
df_train9 = pd.read_csv('../input/mayo-clinic-1024-jpg-part9/train_with_groups.csv')
df_train10 = pd.read_csv('../input/mayo-clinic-1024-jpg-part10/train_with_groups.csv')

df_train1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_id    754 non-null    object 
 1   center_id   754 non-null    int64  
 2   patient_id  754 non-null    object 
 3   image_num   754 non-null    int64  
 4   label       754 non-null    object 
 5   file_size   754 non-null    float64
 6   group       754 non-null    int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 41.4+ KB


In [3]:
# define relevant Paths
#train_source_path1 = Path('../input/mayo-clinic-1024-jpg-part1/train')
#train_source_path2 = Path('../input/mayo-clinic-1024-jpg-part2-1/train')
train_source_paths = [Path(i) / 'train' for i in Path('../input').glob('./mayo-clinic-1024*')]
train_dest_path = Path('../working/data')
ce_train_path = Path('../working/data/CE')
laa_train_path = Path('../working/data/LAA')


In [4]:
lr = [1.9333333333333333e-05,
1.866666666666667e-05,
1.8000000000000004e-05,
1.7333333333333336e-05,
1.6666666666666667e-05,
1.6000000000000003e-05,
1.5333333333333334e-05,
1.4666666666666668e-05,
1.4000000000000001e-05]

In [5]:
# only take first 1k images REMOVE IN PRODUCTION
i = 0 
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import ViTFeatureExtractor, ViTForImageClassification
old_model = Path('../input/model-new/model')
huggingface_model = 'google/vit-base-patch16-224-in21k'
for source_path in train_source_paths:
    #setting up enviroment
    # clean working directory
    if os.path.isdir(train_dest_path):
        shutil.rmtree(train_dest_path)
        os.makedirs(train_dest_path)
    else:
        os.makedirs(train_dest_path)
        
    # create one folder for each image class    
    os.mkdir(ce_train_path)
    os.mkdir(laa_train_path)
    
    # set df
    df_train = pd.read_csv(source_path / '../train_with_groups.csv')
    
    for img in list(source_path.glob('./*.jpg')):
        image_id = img.stem[:8]
        image_label = df_train[df_train['image_id'] == image_id].iloc[0]["label"]
        shutil.copyfile(img, train_dest_path / image_label / img.name)
        
    train, test, id2label, label2id = VisionDataset.fromImageFolder('/kaggle/working/data/', balanced=False)
    if i == 0:
        trainer = VisionClassifierTrainer(
            model_name   = "MayoClinicModel",
            train        =  train,
            test         =  test,
            output_dir   = "../working/out/",
            max_epochs   = 1,
            batch_size   = 32, 
            lr       = 2e-5,
            fp16     = False,
            model = ViTForImageClassification.from_pretrained(
                old_model,
                num_labels = len(label2id),
                label2id   = label2id,
                id2label   = id2label
            ),
            feature_extractor = ViTFeatureExtractor.from_pretrained(huggingface_model),
        )
        i = 1
    else:
        current_model =max(glob.glob(os.path.join('../working/out/MAYOCLINICMODEL/', '*/')), key=os.path.getmtime)
        model_path_source = Path('kaggle/working/out/MAYOCLINICMODEL').glob( '../model')
        trainer = VisionClassifierTrainer(
            model_name   = "MayoClinicModel",
            train        =  train,
            test         =  test,
            output_dir   = "../working/out/",
            max_epochs   = 1,
            batch_size   = 32, 
            lr       = lr[i-1],
            fp16     = False,
            model = ViTForImageClassification.from_pretrained(
                current_model +  'model',
                num_labels = len(label2id),
                label2id   = label2id,
                id2label   = id2label
            ),
            feature_extractor = ViTFeatureExtractor.from_pretrained(huggingface_model),
        )
        i += 1
    

Split Datasets...
train_ds:  52331
+---------+-------+-------+-------+
| Dataset |  CE   |  LAA  | Total |
+---------+-------+-------+-------+
|  Train  | 28535 | 23796 | 52331 |
|  Test   | 5059  | 4177  | 9236  |
+---------+-------+-------+-------+


Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


{'0': 'CE', '1': 'LAA'}
{'CE': '0', 'LAA': '1'}


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 52331
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1636


...

All model checkpoint weights were used when initializing ViTForImageClassification.

All the weights of ViTForImageClassification were initialized from the model checkpoint at ../working/out/MAYOCLINICMODEL/1_2022-08-03-15-59-40/model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ViTForImageClassification for predictions without further training.
loading feature extractor configuration file https://huggingface.co/google/vit-base-patch16-224-in21k/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/7c7f3e780b30eeeacd3962294e5154788caa6d9aa555ed6d5c2f0d2c485eba18.c322cbf30b69973d5aae6c0866f5cba198b5fe51a2fe259d2a506827ec6274bc
Feature extractor ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "size": 224
}

PyTorch

{'0': 'CE', '1': 'LAA'}
{'CE': '0', 'LAA': '1'}
Trainer builded!
Start Training!


Epoch,Training Loss,Validation Loss
1,0.387000,0.371046


***** Running Evaluation *****
  Num examples = 7086
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ../working/out/MAYOCLINICMODEL/1_2022-08-03-16-56-36/trainer/
Configuration saved in ../working/out/MAYOCLINICMODEL/1_2022-08-03-16-56-36/trainer/config.json
Model weights saved in ../working/out/MAYOCLINICMODEL/1_2022-08-03-16-56-36/trainer/pytorch_model.bin
Configuration saved in ../working/out/MAYOCLINICMODEL/1_2022-08-03-16-56-36/model/config.json
Model weights saved in ../working/out/MAYOCLINICMODEL/1_2022-08-03-16-56-36/model/pytorch_model.bin
Feature extractor saved in ../working/out/MAYOCLINICMODEL/1_2022-08-03-16-56-36/feature_extractor/preprocessor_config.json


Model saved at: ../working/out/MAYOCLINICMODEL/1_2022-08-03-16-56-36
